## A. Configuración General.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#2. Constantes.
%run "../constantes.ipynb"

dataset_con_fe = dataset_file_fe_all_1_limpieza

cantidad_meses_train = "all"
ventana = 1

mes_train = mes_train_all_menos_1
mes_test = mes_test

In [ ]:
#3. Funciones
%run "../funciones.ipynb"

In [ ]:
#4. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)
#iii. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#iv. Divido entre Train y Test.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

In [ ]:
#v. Undersampleo.
#a. Filtramos las clases.
continua_train = train_data[(train_data['clase_binaria2'] == 0)]
baja_train = train_data[(train_data['clase_binaria2'] == 1)]

#b. Lista para almacenar los datos submuestreados.
continua_undersampleados = []

#c. Iteramos sobre cada mes para submuestrear.
for mes in continua_train['foto_mes'].unique():
    #1. Filtramos los datos de la clase mayoritaria para el mes específico.
    continua_mes_seleccionado = continua_train[continua_train['foto_mes'] == mes]
    
    #2. Calculamos el tamaño objetivo para el undersampling (30%).
    n_samples = int(len(continua_mes_seleccionado) * 0.3)
    
    #3. Submuestreamos las instancias de la clase mayoritaria para este mes
    continua_mes_seleccionado_undersampleados = resample(continua_mes_seleccionado, 
                                        replace=False, 
                                        n_samples=n_samples, 
                                        random_state=semillas[0])
    
    #4. Agregamos los datos submuestreados a la lista
    continua_undersampleados.append(continua_mes_seleccionado_undersampleados)

#d. Concatenamos todos los meses submuestreados en un solo DataFrame
continua_undersampleados = pd.concat(continua_undersampleados)

#e. Concatenamos la clase mayoritaria submuestreada con la clase minoritaria completa
train_undersampleado = pd.concat([continua_undersampleados, baja_train])

In [ ]:
#vi. Separar en X e y después del undersampling.
#a. Datos para optimizar Optuna.
X_train_undersampleado = train_undersampleado.drop(['clase_ternaria', 'clase_peso', 'clase_binaria2'], axis=1)
y_train_binaria2_undersampleado = train_undersampleado['clase_binaria2']
w_train_undersampleado = train_undersampleado['clase_peso']

#b. Datos para entrenar todo el modelo final para Kaggle.
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

#c. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)

## B. Optimización Hiperparámetros (OH) con cantidad_meses_train meses con df -ventana con ratios incluidos.

In [ ]:
#1. Funcion de optimización de hiperparámetros.
def objective(trial): 
    # Rango de parámetros a buscar sus valores óptimos.
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3) # mas bajo, más iteraciones necesita.
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 15, 900)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)


    # Parámetros que le voy a pasar al modelo.
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    # Creo el dataset para Light GBM.
    train_data_ob = lgb.Dataset(X_train_undersampleado,
                              label=y_train_binaria2_undersampleado, # eligir la clase
                              weight=w_train_undersampleado)
    
    # Entreno.
    cv_results = lgb.cv(
        params,
        train_data_ob,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior
        callbacks=[lgb.early_stopping(int(50 + 5 / learning_rate))],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    # Calculo la ganancia máxima y la mejor iteración donde se obtuvo dicha ganancia.
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

In [ ]:
#2. Voy a realizar un estudio de Optuna para encontrar los mejores parámetros.
#i. Creo la base de datos donde guardar los resultados.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"

study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling_limpieza" # Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting.

#ii. Creo el estudio.
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

#iii. Corro el estudio.
study.optimize(objective, n_trials=100)

In [10]:
#4. Visualizo los resultados del estudio, para modificar los rangos de análisis.

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

In [ ]:
study.best_trial.params

## C. Entrenamiento con mejos Hiperparámetros.

In [ ]:
#1. Tomamos el mejor modelo y con eso entrenamos todos los datos.
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data_modelo = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model_lgb = lgb.train(params,
                  train_data_modelo,
                  num_boost_round=best_iter)

In [ ]:
#2. Observamos las variables más importantes para el modelo.
#i. Gráfico.
lgb.plot_importance(model_lgb, figsize=(10, 20))
plt.show()

In [ ]:
#ii. Dataframe.
#a. Extract feature importance and feature names.
importance = model_lgb.feature_importance()
features = model_lgb.feature_name()

#b. Create a dataframe for better visualization.
importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})

#c. Sort by importance in descending order.
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

#d. show.
importance_df.head(100)

## D. Guardamos el modelo entrenado.

In [ ]:
#1. Guardamos el modelo.
# Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting, tercer dígito número de entrenamiento.
model_lgb.save_model(modelos_path + 'lgbm_{}_{}_undersampling_limpieza.txt'.format(cantidad_meses_train,ventana))